In [402]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

import datetime
from scipy import stats as st

font = {'family': 'sans-serif',
        'color':  '#a72693',
        'weight': 'normal',
        'size': 20,
        }

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

df = pd.read_csv('data_6.csv', delimiter ='\t')
df = df.drop('Unnamed: 0', 1)

df.set_axis(['date', 'user_id', 'product_id', 'is_added_to_cart', 'is_cart_viewed', 'is_ordered'],
         axis = 'columns', inplace = True)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


In [403]:
#df.info()
#df.columns
#df.isnull().sum()
#df.duplicated().sum()
df.head(50)

,date,user_id,product_id,is_added_to_cart,is_cart_viewed,is_ordered
0,2020-09-01,99212,186.00,False,NaN,NaN
1,2020-09-01,99212,21.00,True,NaN,NaN
2,2020-09-01,99212,41.00,False,NaN,NaN
3,2020-09-01,99212,129.00,False,NaN,NaN
4,2020-09-01,99212,99.00,True,NaN,NaN
5,2020-09-01,99212,26.00,True,NaN,NaN
6,2020-09-01,99212,90.00,True,NaN,NaN
7,2020-09-01,99212,170.00,False,NaN,NaN
8,2020-09-01,99212,112.00,False,NaN,NaN
9,2020-09-01,99212,39.00,False,NaN,NaN


In [404]:
# просмотр товара и добавление товара в корзину
#df.loc[pd.isna(df['is_cart_viewed']) & pd.isna(df['is_ordered']), 'event'] = 'shown_n_add' 

# просмотр корзины и покупка
#df.loc[pd.isna(df['product_id']), 'event'] = 'cartviewed_n_bought' 

# добавление в корзину
df.loc[df['is_added_to_cart'] == False, 'event'] = 'product_viewed'

# добавление в корзину
df.loc[df['is_added_to_cart'] == True, 'event'] = 'added_to_cart'

# просмотр корзины 
df.loc[df['is_cart_viewed'] == True, 'event'] = 'cart_viewed'

# покупка
df.loc[df['is_ordered'] == True, 'event'] = 'bought'


#df[df['event'] == 'cartviewed_n_bought'].count()               # посмотрели корзину и купили
#df[df['event'] == 'bought'].count()                            # купили
#df[(df['event'] == 'cart_viewed') & (df['event'] == 'cartviewed_n_bought')].count()# пересечений нет
#df[(df['event'] == 'bought') & (df['event'] == 'cartviewed_n_bought')].count()# пересечений нет
#df[(df['event'] == 'bought') | (df['event'] == 'cartviewed_n_bought')].count()# всего покупок

pd.value_counts(df['event'])                                # действия пользователей
#df.groupby('event')['user_id'].agg('count')
#len(df['user_id'].value_counts())                              # количество пользователей

product_viewed    119590
added_to_cart      72441
cart_viewed         5863
bought              1387
Name: event, dtype: int64

**Полная воронка**

In [405]:
df1 = dict(pd.value_counts(df['event']))
data = dict(
    event=list(df1.keys()),
    count=list(df1.values()))

fig = go.Figure(go.Funnel(x=data['count'], 
                          y=data['event'],
                          textinfo = "value+percent initial", 
                          marker = {"color": "#1c1847"})
               )
fig.show()

#.groupby(['event','first_event'])['event','first_event'].agg({'first_event':'count'})

**Воронка по неделям**

In [427]:
first_event = df.groupby(by = 'user_id', as_index=False)['date'].agg({'first_event': 'min'})
clients_first_event = pd.merge(df, first_event, 
                                    how='left', 
                                    on='user_id')

first_event_abs = clients_first_event['date'].min()
last_visit = clients_first_event['first_event'].max()
kohort = {'count' : None,
          'date' : None}
kohort_date = []
kohort_count = []
while last_visit > first_event_abs :
    
    cache = clients_first_event[(
        clients_first_event['first_event'] >= first_event_abs
        ) & (
        clients_first_event['first_event'] < first_event_abs + datetime.timedelta(days=7)
        )]
    
    df1 = dict(pd.value_counts(cache['event']))
    data = dict(
        event=list(df1.keys()),
        count=list(df1.values()))
    
    fig = go.Figure(go.Funnel(x=data['count'], 
                              y=data['event'],
                              textinfo = "value+percent initial", 
                              marker = {"color": "#1c1847"})
                   )
    kohort_date.append(first_event_abs)
    kohort_count.append(df1['product_viewed'])
    fig.show()
    first_event_abs += datetime.timedelta(days=7)

,0,1,2,3,4,5,6,7,8
0,16321,14965,14611,14193,13746,13106,12802,12279,7567
1,2020-09-01 00:00:00,2020-09-08 00:00:00,2020-09-15 00:00:00,2020-09-22 00:00:00,2020-09-29 00:00:00,2020-10-06 00:00:00,2020-10-13 00:00:00,2020-10-20 00:00:00,2020-10-27 00:00:00


mix = pd.merge(df, first_event, how='left', on=['user_id'])
mix = mix.drop(['product_id', 'is_added_to_cart', 'is_cart_viewed', 'is_ordered'], 1)
mix.columns
preresult = mix.groupby(by=['date', 'user_id', 'first_event','event'], as_index=False)
preresult = preresult[preresult['event'] == 'bought']
preresult.head(50)
preresult = preresult.agg({ 'event': { 'total': 'sum', 'ordersCount': 'count' } })

In [438]:
kohort = pd.DataFrame(data = [kohort_count, kohort_date])
kohort = kohort.transpose()
kohort.columns = ['count_of_sales', 'date']
fig = px.bar(kohort, x='date', y='count_of_sales')
fig.show()